# #2 Intro to Python

_Let's scale Jupyter to the window size_

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width 80% : important;}</style>"))

## Let's import some typical libraries to work with...

In [ ]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import gc

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import time
import datetime as dt
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta

import math

import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

from IPython.display import clear_output

But who want to copy-paste libraries all the time...?<br>Luckily they can be **imported from a file**

In [ ]:
from toImportLibraries import *

This can also be done with e.g. functions:

In [ ]:
from toImportFunctions import *

In [ ]:
checkSciLibVersions()

Some iPython Magic to help plots and reload import files if modfied:

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Extending the example:

Same excercise, but
* One of the input is in Excel
* Save the plot as a .png file
* Send an e-mail of the results

check if the file we need exists

In [ ]:
os.path.exists('./ID_amt_map.xlsx')

Or see the content of the folder

In [ ]:
os.listdir('./')

Read the csv as last time

In [ ]:
dfNames = pd.read_csv("ID_name_map.csv")
dfNames.head()

Read the Excel file

In [ ]:
xlFile = pd.ExcelFile('./ID_amt_map.xlsx')

In [ ]:
#Parse the worksheet with the data we need
dfAmt = xlFile.parse('data')
dfAmt.head()

Merge the data

In [ ]:
dfFinal = pd.merge(dfNames, dfAmt, how='inner', on='ID')
dfFinal.head()

Save results in a new file

In [ ]:
dfFinal.to_csv('Herbs.csv', index=False)

Create a new plot and save it

In [ ]:
with plt.xkcd(): #https://matplotlib.org/stable/gallery/showcase/xkcd.html
    fig = plt.figure(figsize=(14, 6))
    ax = dfFinal['Amount'].plot(kind='bar')
    ax.set_xticklabels(dfFinal['Name'])
    ax.set_title('Herb Stock')
    ax.set_ylabel('Amount')
    plt.savefig('herbStock.png', dpi=100, bbox_inches='tight')
    plt.show()

Let's send it to someone. The adressee can be given by input:

In [ ]:
emailReceiver = input("e-mail address to send mail to: ")

In [ ]:
print(emailReceiver)

Send the mail with [smtplib](https://docs.python.org/3/library/smtplib.html#smtp-example)<br>
Here I use a Gmail address to send mails from. Don't forget to enable less secure apps for this account to let python access. 

In [ ]:
# Not a good way to store passwords...
passwordForSender = input("Type your password for sender e-mail address and press enter:")
clear_output()

In [ ]:
emailSubject = "Low Herb Stock"
emailSender = "<someEmailAddress>@gmail.com"

message = MIMEMultipart("alternative")
message["Subject"] = emailSubject
message["From"] = emailSender
message["To"] = emailReceiver

# Create the plain-text and HTML version of your message
html = """\
<html>
  <body>
    <p>Hi,<br>
    Please fill up the Herb Stock before the first Potion class of next semester.<br>
    </p>
  </body>
</html>
"""

# Turn these into plain/html MIMEText objects
text = MIMEText(html, "html")

# Add HTML/plain-text parts to MIMEMultipart message
# The email client will try to render the last part first
message.attach(text)

imageFile = "herbStock.png"
img_data = open(imageFile, 'rb').read()
image = MIMEImage(img_data, name=(imageFile))
message.attach(image)

# Create secure connection with server and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(emailSender, passwordForSender)
    server.sendmail(
        emailSender, emailReceiver, message.as_string()
    )
    server.quit()
    
print("> Message sent")

#### Even better: let's just run this from Windows by a simple double-click
0. Add Python to Windows Environment Variables (if you don't have it already)
1. Create a test.py file from the code above (File/download or e.g. merge cells and copy-paste...)
2. Copy the followings to a test.cmd file and place it next to the test.py file: <br>
python test.py 

## Connecting to SQL Databases
The general approach using pandas.io.sql:
- prepare sql query as string
- connect DB
- pull data with read_sql to a DataFrame
- close connection

Most Databases have their own libraries for Python connection. <br> E.g.:
- [IBM](https://www.ibm.com/docs/en/db2/11.1?topic=db-connecting-database-server)
- [Oracle](https://www.oracletutorial.com/python-oracle/connecting-to-oracle-database-in-python/)
- [Sybase](https://wiki.python.org/moin/Sybase)

## Date - Time - Datetime

In [ ]:
print(date.today())
date.today()

In [ ]:
today = date.today()
print(today.year, today.month, today.day)

In [ ]:
print(datetime.now())

__For formatting: [https://strftime.org/](https://strftime.org/)__

In [ ]:
now = datetime.now()
print(type(now), now)

In [ ]:
nowStr = now.strftime('%Y/%m/%d')
print(type(nowStr), nowStr)

In [ ]:
print(now.strftime('Current time: %I:%M:%S %p'))
print(now.strftime('24-hour time: %H:%M'))

__Ellapsed time: timedelta__

In [ ]:
#Simple previous working day:
dayDelay = 1
cop = date.today().strftime('%Y-%m-%d')
if date.today().strftime('%a') == 'Mon':
    dayDelay = 3
elif date.today().strftime('%a') == 'Sun':
    dayDelay = 2
copPrev = (date.today() - dt.timedelta(dayDelay)).strftime('%Y-%m-%d')
print('Last COP\t today')
print(copPrev, '\t', cop)
print(type(copPrev), '\t',type(cop))

In [ ]:
#Difference between dates in days
eoy = date(today.year,12,31)
print(today, '->', eoy)
print((eoy-today).days, 'days left from the year.')

__Dates with specific frequency__ - using pandas<br>
[https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases)

In [ ]:
#E.g. Business Month Starts
pd.date_range(today, eoy, freq = 'BMS')

## Filter file line-by-line

In [ ]:
fileName = 'financialSample.csv'

In [ ]:
def findSales(fileName,Country):
    with open('filteredFile.csv', 'w+') as writeFile:
        writeFile.write('Segment,Country,Product\n')
        
        with open(fileName,'r') as readFile:
            for i,line in enumerate(readFile):
                
                if (i % 100) == 0:
                    print(str(i/100)+'e2',end=', ')
                    
                lineSplit = line.split('\n')[0]
                lineSplit = lineSplit.split(';')
                
                if lineSplit[1] == Country:
                    writeFile.write(','.join([lineSplit[0], lineSplit[1], lineSplit[2]])+'\n')
                    
    writeFile.close()
    
    print('\nTotal Rows investigated:',i)
                

In [ ]:
findSales(fileName,'Germany')

In [ ]:
df = pd.read_csv('filteredFile.csv')
df.head()